# Train
This notebook performs model training.
To see the hyperparameter-tuning, use the **bci-avm-dask** cluster with the **Hypertuning** notebook.

<p align="center">
<img width=25% src="https://blockchainclimate.org/wp-content/uploads/2020/11/cropped-BCI_Logo_LR-400x333.png" alt="bciAVM" height="300"/>
</p>

[![PyPI](https://badge.fury.io/py/bciavm.svg?maxAge=2592000)](https://badge.fury.io/py/bciavm)
[![PyPI Stats](https://img.shields.io/badge/bciavm-avm-blue)](https://pypistats.org/packages/bciavm)


This notebook contains code to take a `mlflow` registered model and distribute its work with a `Dask` cluster. 
<table>
    <tr>
        <td>
            <img width=25% src="https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/dask.png" width="150">
        </td>
    </tr>
</table>

The [Blockchain Climate Institute](https://blockchainclimate.org) (BCI) is a progressive think tank providing leading expertise in the deployment of emerging technologies for climate and sustainability actions. 

As an international network of scientific and technological experts, BCI is at the forefront of innovative efforts, enabling technology transfers, to create a sustainable and clean global future.

# Automated Valuation Model (AVM) 

### About
AVM is a term for a service that uses mathematical modeling combined with databases of existing properties and transactions to calculate real estate values. 
The majority of automated valuation models (AVMs) compare the values of similar properties at the same point in time. 
Many appraisers, and even Wall Street institutions, use this type of model to value residential properties. (see [What is an AVM](https://www.investopedia.com/terms/a/automated-valuation-model.asp) Investopedia.com)

For more detailed info about the AVM, please read the **About** paper found here `resources/2021-BCI-AVM-About.pdf`.

### Valuation Process
<img src="resources/valuation_process.png" height="360" >

**Key Functionality**

* **Supervised algorithms** 
* **Tree-based & deep learning algorithms** 
* **Feature engineering derived from small clusters of similar properties** 
* **Ensemble (value blending) approaches** 

### Set the required AWS Environment Variables
```shell
export ACCESS_KEY=YOURACCESS_KEY
export SECRET_KEY=YOURSECRET_KEY
export BUCKET_NAME=bci-transition-risk-data
export TABLE_DIRECTORY=/dbfs/FileStore/tables/
```

### Next Steps
Read more about bciAVM on our [documentation page](https://blockchainclimate.org/thought-leadership/#blog):

### How does it relate to BCI Risk Modeling?
<img src="resources/bci_flowchart_2.png" height="280" >


### Technical & financial support for development provided by:
<a href="https://www.gcode.ai">
    <img width=15% src="https://staticfiles-img.s3.amazonaws.com/avm/gcode_logo.png" alt="GCODE.ai"  height="25"/>
</a>


### Install [from PyPI](https://pypi.org/project/bciavm/)
```shell
pip install bciavm
```

This notebook covers the following steps:
- Import data from your local machine into the Databricks File System (DBFS)
- Download data from s3
- Train a machine learning models (or more technically, multiple models in a stacked pipeline) on the dataset
- Register the model in MLflow

In [2]:
import os
import time
import uuid
from datetime import datetime
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask
from dask.distributed import Client
import mlflow
from mlflow.tracking import MlflowClient
import bciavm
import re
from urllib.request import urlopen
import zipfile
from io import BytesIO
import io
from bciavm.core.config import your_bucket
from bciavm.utils.bci_utils import ReadParquetFile, get_postcodeOutcode_from_postcode, get_postcodeArea_from_outcode, drop_outliers, preprocess_data
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit


2021-06-15 06:11:58,477 featuretools - WARNING Featuretools failed to load plugin nlp_primitives from library nlp_primitives. For a full stack trace, set logging to debug.

In [3]:
MLFLOW_TRACKING_URI = os.environ["MLFLOW_TRACKING_URI"]
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment('/Users/mike.casale@blockchainclimate.org/Experiments/train')

## Importing Data
  
In this section, you download a dataset from the BCI s3 bucket and upload it to Databricks File System (DBFS).

1. In addition to the s3 download, navigate to [https://www.freemaptools.com/download-uk-postcode-lat-lng.html/](https://www.freemaptools.com/download-uk-postcode-lat-lng.html/) and download both `ukpostcodes.csv` and `postcode_outcodes.csv` to your local machine.

2. From this Databricks notebook, select *File* > *Upload Data*, and drag these files to the drag-and-drop target to upload them to the Databricks File System (DBFS). 

3. Click *Next*. Some auto-generated code to load the data appears. Select *pandas*, and copy the example code. 

4. Create a new cell, then paste in the sample code. It will look similar to the code shown in the following cell. Make these changes:

In [5]:
print(your_bucket)

s3.Bucket(name='bci-transition-risk-data')

In [6]:
dfPricesEpc = pd.DataFrame()
dfPrices = pd.DataFrame()

yearArray = ['2020', '2019']
for year in yearArray:

    #NOTE: To connect to s3, ensure your environment variables for AWS are set (see top of notebook for instructions)
    singlePriceEpcFile = pd.DataFrame(ReadParquetFile(your_bucket, 'epc_price_data/byDate/2021-02-04/parquet/' + year))
    dfPricesEpc = dfPricesEpc.append(singlePriceEpcFile)

dfPricesEpc['POSTCODE_OUTCODE'] = dfPricesEpc['Postcode'].apply(get_postcodeOutcode_from_postcode)
dfPricesEpc['POSTCODE_AREA'] = dfPricesEpc['POSTCODE_OUTCODE'].apply(get_postcodeArea_from_outcode)
dfPricesEpc.groupby('TypeOfMatching_m').count()['Postcode']
dfPricesEpc = dfPricesEpc.rename({'Postcode':'POSTCODE'},axis=1)

## Preprocessing Data
Prior to training a model, check for missing values and split the data into training and validation sets.

In [8]:
X_train, X_test, y_train, y_test = bciavm.utils.bci_utils.preprocess_data(dfPricesEpc)

In [9]:
try: os.mkdir('/dbfs/FileStore/tables/avm')
except: pass

#NOTE: We need to log these to ensure our scoring process does not cheat
X_train.to_csv('/dbfs/FileStore/tables/avm/X_train.csv',index=False)
X_test.to_csv('/dbfs/FileStore/tables/avm/X_test.csv',index=False)
y_train.to_csv('/dbfs/FileStore/tables/avm/y_train.csv',index=False)
y_test.to_csv('/dbfs/FileStore/tables/avm/y_test.csv',index=False)

In [10]:
import mlflow
mlflow.set_experiment('/Users/mike.casale@blockchainclimate.org/Experiments/tune')
best_run = mlflow.search_runs(order_by=['metrics.MdAPE ASC']).iloc[0]
print(f'MdAPE of Best Run: {best_run["metrics.MdAPE"]}')

MdAPE of Best Run: 10.955096633310719

In [11]:
best_run = best_run.reset_index()
best_run

Out[8]:

,index,0
0,run_id,fceccd669e204174b04494af2d85d77d
1,experiment_id,1742618768450250
2,status,FINISHED
3,artifact_uri,dbfs:/databricks/mlflow-tracking/1742618768450...
4,start_time,2021-06-14 22:26:09.706000+00:00
...,...,...
101,tags.estimator_name,None
102,tags.mlflow.databricks.cluster.id,None
103,tags.mlflow.databricks.cluster.libraries,None
104,tags.mlflow.runName,None


In [12]:
params = {}
for col in best_run['index'].values:
  if 'params' in col:
    params[col]=best_run[best_run['index']==col][0].values[0]

In [13]:
params

Out[11]: {'params.Imputer_categorical_impute_strategy': 'most_frequent',
 'params.One Hot Encoder_handle_missing': 'error',
 'params.One Hot Encoder_top_n': '2',
 'params.XGB Regressor_n_estimators': '260',
 'params.XGB Regressor_min_child_weight': '6',
 'params.One Hot Encoder_drop': 'None',
 'params.XGB Regressor_reg_alpha': '0.04216890859017286',
 'params.XGB Regressor_learning_rate': '0.1040805594257481',
 'params.Imputer_numeric_fill_value': 'None',
 'params.One Hot Encoder_categories': 'None',
 'params.One Hot Encoder_features_to_encode': "['agg_cat']",
 'params.One Hot Encoder_handle_unknown': 'ignore',
 'params.Imputer_numeric_impute_strategy': 'median',
 'params.Imputer_categorical_fill_value': 'None',
 'params.XGB Regressor_max_depth': '11',
 'params.XGB Regressor_reg_lambda': '0.2541818897259287',
 'params.validation_fraction': None,
 'params.hidden_layer_sizes': None,
 'params.verbose': None,
 'params.learning_rate_init': None,
 'params.ML Perceptron Regressor_beta_1': None,
 'params.epsilon': None,
 'params.Final Estimator_reg_lambda': None,
 'params.alpha': None,
 'params.Final Estimator_reg_alpha': None,
 'params.Final Estimator_min_child_weight': None,
 'params.algorithm': None,
 'params.ML Perceptron Regressor_max_iter': None,
 'params.ML Perceptron Regressor_batch_size': None,
 'params.learning_rate': None,
 'params.metric_params': None,
 'params.ML Perceptron Regressor_n_iter_no_change': None,
 'params.power_t': None,
 'params.K Nearest Neighbors_algorithm': None,
 'params.activation': None,
 'params.Final Estimator_n_estimators': None,
 'params.ML Perceptron Regressor_learning_rate_init': None,
 'params.metric': None,
 'params.n_jobs': None,
 'params.ML Perceptron Regressor_alpha': None,
 'params.K Nearest Neighbors_p': None,
 'params.tol': None,
 'params.leaf_size': None,
 'params.weights': None,
 'params.K Nearest Neighbors_metric': None,
 'params.beta_2': None,
 'params.K Nearest Neighbors_weights': None,
 'params.ML Perceptron Regressor_learning_rate': None,
 'params.p': None,
 'params.max_fun': None,
 'params.beta_1': None,
 'params.ML Perceptron Regressor_activation': None,
 'params.ML Perceptron Regressor_beta_2': None,
 'params.batch_size': None,
 'params.momentum': None,
 'params.K Nearest Neighbors_n_jobs': None,
 'params.max_iter': None,
 'params.Final Estimator_learning_rate': None,
 'params.K Nearest Neighbors_n_neighbors': None,
 'params.random_state': None,
 'params.K Nearest Neighbors_leaf_size': None,
 'params.n_iter_no_change': None,
 'params.early_stopping': None,
 'params.warm_start': None,
 'params.solver': None,
 'params.Final Estimator_max_depth': None,
 'params.ML Perceptron Regressor_solver': None,
 'params.n_neighbors': None,
 'params.ML Perceptron Regressor_epsilon': None,
 'params.ML Perceptron Regressor_early_stopping': None,
 'params.shuffle': None,
 'params.nesterovs_momentum': None}

In [14]:
from bciavm.pipelines import RegressionPipeline

class Pipeline(RegressionPipeline):
    custom_name = 'AVM Pipeline'
    component_graph = {  'Preprocess Transformer': [
                             'Preprocess Transformer'
                         ],
                         'Imputer': [
                             'Imputer', 
                             'Preprocess Transformer'
                         ],
                         'One Hot Encoder': [
                             'One Hot Encoder', 
                             'Imputer'
                         ],
                         'XGB Regressor': [
                             'XGBoost Regressor', 
                             'One Hot Encoder'
                         ]}

#Uses the best params from the Tune-Hyperparameters Notebook
parameters = {'Imputer': {'categorical_impute_strategy': 'most_frequent',
                'numeric_impute_strategy': params['params.Imputer_numeric_impute_strategy'],
                'categorical_fill_value': None,
                'numeric_fill_value': None,
              },
             'One Hot Encoder': {'top_n': int(params['params.One Hot Encoder_top_n']),
                'features_to_encode': ['agg_cat'],
                'categories': None,
                'drop': None,
                'handle_unknown': 'ignore',
                'handle_missing': 'error',
              },
             'XGB Regressor': {'learning_rate': float(params['params.XGB Regressor_learning_rate']),
                        'max_depth': int(params['params.XGB Regressor_max_depth']),
                        'min_child_weight': int(params['params.XGB Regressor_min_child_weight']),
                        'reg_alpha': float(params['params.XGB Regressor_reg_alpha']),
                        'reg_lambda': float(params['params.XGB Regressor_reg_lambda']),
                        'n_estimators': int(params['params.XGB Regressor_n_estimators'])
             }
}


avm_pipeline = Pipeline(parameters=parameters)
avm_pipeline

Out[18]: Pipeline(parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'median', 'categorical_fill_value': None, 'numeric_fill_value': None, 'random_seed': 0}, 'One Hot Encoder':{'top_n': 2, 'features_to_encode': ['agg_cat'], 'categories': None, 'drop': None, 'handle_unknown': 'ignore', 'handle_missing': 'error', 'random_seed': 0}, 'XGB Regressor':{'learning_rate': 0.1040805594257481, 'max_depth': 11, 'min_child_weight': 6, 'reg_alpha': 0.04216890859017286, 'reg_lambda': 0.2541818897259287, 'n_estimators': 260},})

In [15]:
max(y_train), min(y_train)

Out[19]: (1100000.0, 37750.0)

In [16]:
bciavm.__version__

Out[20]: '1.21.25'

In [17]:
avm_pipeline.fit(X_train, y_train)

Out[21]: Pipeline(parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'median', 'categorical_fill_value': None, 'numeric_fill_value': None, 'random_seed': 0}, 'One Hot Encoder':{'top_n': 2, 'features_to_encode': ['agg_cat'], 'categories': None, 'drop': None, 'handle_unknown': 'ignore', 'handle_missing': 'error', 'random_seed': 0}, 'XGB Regressor':{'learning_rate': 0.1040805594257481, 'max_depth': 11, 'min_child_weight': 6, 'reg_alpha': 0.04216890859017286, 'reg_lambda': 0.2541818897259287, 'n_estimators': 260},})

In [18]:
bciavm.__version__

Out[22]: '1.21.25'

In [19]:
try: os.mkdir('/dbfs/FileStore/artifacts')
except: pass

avm_pipeline.save('/dbfs/FileStore/artifacts/avm_pipeline_'+str(bciavm.__version__)+'.pkl')
model = avm_pipeline

In [20]:
scores = model.score(X_test, y_test, objectives = ['MAPE',
                                                   'MdAPE',
                                                   'ExpVariance',
                                                   'MaxError',
                                                   'MedianAE',
                                                   'MSE',
                                                   'MAE',
                                                   'R2',
                                                   'Root Mean Squared Error'])
scores

Out[24]: OrderedDict([('MAPE', 15.426957110849646),
 ('MdAPE', 9.454798377403845),
 ('ExpVariance', 0.892766410427703),
 ('MaxError', 920230.5625),
 ('MedianAE', 20012.52734375),
 ('MSE', 3182615684.8227105),
 ('MAE', 33918.256877698455),
 ('R2', 0.8927647074444016),
 ('Root Mean Squared Error', 56414.67614745928)])

In [21]:
input_example = X_test.dropna().sample(1)

In [22]:
input_example.dtypes

Out[27]: unit_indx Int64
POSTCODE category
POSTCODE_OUTCODE category
POSTTOWN_e category
PROPERTY_TYPE_e category
TOTAL_FLOOR_AREA_e float64
NUMBER_HEATED_ROOMS_e float64
FLOOR_LEVEL_e float64
Latitude_m float64
Longitude_m float64
POSTCODE_AREA category
dtype: object

In [23]:
avm_pipeline.predict(input_example)

Out[28]: 0 206913.765625
Name: Price_p, dtype: float64

In [24]:
avm_pipeline.avm(input_example, batch_sample_sets=1, pred_interval=.9,  min_sample_sz=15)

Out[29]:

,unit_index,avm,avm_lower,avm_upper,conf,ts,latest_production_version,latest_staging_version
0,494459,206914.0,163729.0,264932.0,0.719602,1623738380.364888,29,NaN


In [25]:
class ConfModel(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        self.model = model

    def predict(self, X):
        return self.model.avm(X, pred_interval=.9,  min_sample_sz=15)

In [26]:
conf_model = ConfModel(avm_pipeline)
conf_model.predict(input_example)

Out[40]:

,unit_index,avm,avm_lower,avm_upper,conf,ts,latest_production_version,latest_staging_version
0,494459,206914.0,171142.0,242595.0,0.827118,1623740900.527656,29,NaN


In [27]:
mlflow.end_run()
mlflow.set_experiment('/Users/mike.casale@blockchainclimate.org/Experiments/train')
with mlflow.start_run(run_name='avm') as run:
    objectives = [ 'MAPE',
                   'MdAPE',
                   'ExpVariance',
                   'MaxError',
                   'MedianAE',
                   'MSE',
                   'MAE',
                   'R2',
                   'Root Mean Squared Error']
    
    for o in objectives:
        mlflow.log_metric(o, scores[o])
    
    #log the training/testing data
    mlflow.log_artifact('/dbfs/FileStore/tables/avm/X_train.csv')
    mlflow.log_artifact('/dbfs/FileStore/tables/avm/X_test.csv')
    mlflow.log_artifact('/dbfs/FileStore/tables/avm/y_train.csv')
    mlflow.log_artifact('/dbfs/FileStore/tables/avm/y_test.csv')
    
    #log the raw model/pipeline artifact
    mlflow.log_artifact('/dbfs/FileStore/artifacts/avm_pipeline_'+str(bciavm.__version__)+'.pkl')
    
    #log the mlflow.sklearn flavor model with input example
    mlflow.sklearn.log_model(
                             avm_pipeline,
                             "model", 
                             input_example=input_example,
                            )
    


In [28]:
from mlflow.tracking import MlflowClient


mlflow.end_run()

model_name='avm'
model_version = mlflow.register_model(f"runs:/{run.info.run_id}/model", model_name)

client = MlflowClient()

try:
  # Promote the new model version to Production
  client.transition_model_version_stage(
    name=model_name,
    version=int(model_version.version) - 1,
    stage="archived"
  )
except: pass

# Promote the new model version to Production
client.transition_model_version_stage(
  name=model_name,
  version=model_version.version,
  stage="Production"
)

Registered model 'avm' already exists. Creating a new version of this model...
2021/06/15 06:26:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: avm, version 29
Created version '29' of model 'avm'.
Out[33]: <ModelVersion: creation_timestamp=1623738413736, current_stage='Production', description='', last_updated_timestamp=1623738429390, name='avm', run_id='f064d753541e4326b3f402ba55311b33', run_link='', source='dbfs:/databricks/mlflow-tracking/1002465070968293/f064d753541e4326b3f402ba55311b33/artifacts/model', status='READY', status_message='', tags={}, user_id='497226190874911', version='29'>

In [29]:
# conf_model = Model(model)

mlflow.end_run()
with mlflow.start_run(run_name='avm-conf') as run:
    objectives = [ 'MAPE',
                   'MdAPE',
                   'ExpVariance',
                   'MaxError',
                   'MedianAE',
                   'MSE',
                   'MAE',
                   'R2',
                   'Root Mean Squared Error']
    
    for o in objectives:
        mlflow.log_metric(o, scores[o])
    
    #log the training/testing data
    mlflow.log_artifact('/dbfs/FileStore/tables/avm/X_train.csv')
    mlflow.log_artifact('/dbfs/FileStore/tables/avm/X_test.csv')
    mlflow.log_artifact('/dbfs/FileStore/tables/avm/y_train.csv')
    mlflow.log_artifact('/dbfs/FileStore/tables/avm/y_test.csv')
    
    mlflow.log_params({'max_sample_sz':100, 'min_sample_sz':15, 'pred_interval':.9, 'conf_min':.5})
    
    #log the raw model/pipeline artifact
    mlflow.log_artifact('/dbfs/FileStore/artifacts/avm_pipeline_'+str(bciavm.__version__)+'.pkl')
    
    #log the mlflow.sklearn flavor model with input example
    mlflow.sklearn.log_model(
                             conf_model,
                             "model", 
                             input_example=input_example,
                            )
    



In [30]:
from mlflow.tracking import MlflowClient


mlflow.end_run()

model_name='avm-conf'
model_version = mlflow.register_model(f"runs:/{run.info.run_id}/model", model_name)

client = MlflowClient()

try:
  # Promote the new model version to Production
  client.transition_model_version_stage(
    name=model_name,
    version=int(model_version.version) - 1,
    stage="archived"
  )
except: pass

# Promote the new model version to Production
client.transition_model_version_stage(
  name=model_name,
  version=model_version.version,
  stage="Production"
)

Registered model 'avm-conf' already exists. Creating a new version of this model...
2021/06/15 07:09:13 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: avm-conf, version 14
Created version '14' of model 'avm-conf'.
Out[42]: <ModelVersion: creation_timestamp=1623740953381, current_stage='Production', description='', last_updated_timestamp=1623740969194, name='avm-conf', run_id='a4e0b175db3b4eb69e92c1112f61d5fd', run_link='', source='dbfs:/databricks/mlflow-tracking/1002465070968293/a4e0b175db3b4eb69e92c1112f61d5fd/artifacts/model', status='READY', status_message='', tags={}, user_id='497226190874911', version='14'>

In [31]:
# dbutils.notebook.run("/Users/mike.casale@blockchainclimate.org/Batch Predict", 60)
